<a href="https://colab.research.google.com/github/SorrowAddict/SK-FLY-AI/blob/main/ChatBot/240131_02_%EC%9E%84%EB%B2%A0%EB%94%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 임베딩

### 필요 라이브러리 설치

In [1]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 30.2 MB/s eta 0:00:00


## 원 핫 인코딩

In [2]:
from konlpy.tag import Komoran
import numpy as np

komoran = Komoran()
text = "오늘 날씨는 구름이 많아요."

# 명사만 추출
nouns = komoran.nouns(text)
print(nouns)

# 단어 사전 구축 및 단어별 인덱스 부여
dics = {}
for word in nouns:
    if word not in dics.keys():
        dics[word] = len(dics)
print(dics)

# 원-핫 인코딩
nb_classes = len(dics)
targets = list(dics.values())
one_hot_targets = np.eye(nb_classes)[targets]
print(one_hot_targets)

['오늘', '날씨', '구름']
{'오늘': 0, '날씨': 1, '구름': 2}
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [3]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time


# 네이버 영화 리뷰 데이터 읽어옴
def read_review_data(filename):
    with open(filename, 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] # header 제거
    return data

In [6]:
# 측정 시작
start = time.time()

# 리뷰 파일 읽어오기
print('1) 말뭉치 데이터 읽기 시작')
review_data = read_review_data('./ratings.txt')
print(len(review_data)) # 리뷰 데이터 전체 개수
print('1) 말뭉치 데이터 읽기 완료: ', time.time() - start)

1) 말뭉치 데이터 읽기 시작
200000
1) 말뭉치 데이터 읽기 완료:  0.6028938293457031


In [7]:
# 문장단위로 명사만 추출해 학습 입력 데이터로 만듬
print('2) 형태소에서 명사만 추출 시작')
komoran = Komoran()
docs = [komoran.nouns(sentence[1]) for sentence in review_data]
print('2) 형태소에서 명사만 추출 완료: ', time.time() - start)

2) 형태소에서 명사만 추출 시작
2) 형태소에서 명사만 추출 완료:  167.12847542762756


In [8]:
# word2vec 모델 학습
print('3) word2vec 모델 학습 시작')
model = Word2Vec(sentences=docs, vector_size=200, window=4, min_count=2, sg=1)
print('3) word2vec 모델 학습 완료: ', time.time() - start)

3) word2vec 모델 학습 시작
3) word2vec 모델 학습 완료:  193.45237851142883


In [ ]:
# 모델 저장
print('4) 학습된 모델 저장 시작')
model.save('nvmc.model')
print('4) 학습된 모델 저장 완료: ', time.time() - start)

In [9]:
# 학습된 말뭉치 개수, 코퍼스 내 전체 단어 개수
print("corpus_count : ", model.corpus_count)
print("corpus_total_words : ", model.corpus_total_words)

corpus_count :  200000
corpus_total_words :  1076896


In [10]:
from gensim.models import Word2Vec

# 모델 로딩
model = Word2Vec.load('nvmc.model')
print("corpus_total_words : ", model.corpus_total_words)

# '사랑'이란 단어로 생성한 단어 임베딩 벡터
print('사랑 : ', model.wv['사랑'])

# 단어 유사도 계산
print("일요일 = 월요일\t", model.wv.similarity(w1='일요일', w2='월요일'))
print("안성기 = 배우\t", model.wv.similarity(w1='안성기', w2='배우'))
print("대기업 = 삼성\t", model.wv.similarity(w1='대기업', w2='삼성'))
print("일요일 != 삼성\t", model.wv.similarity(w1='일요일', w2='삼성'))
print("히어로 != 삼성\t", model.wv.similarity(w1='히어로', w2='삼성'))

# 가장 유사한 단어 추출
print(model.wv.most_similar("안성기", topn=5))
print(model.wv.most_similar("시리즈", topn=5))

corpus_total_words :  1076896
사랑 :  [ 8.75288919e-02  3.40068191e-01  4.73868474e-02  3.77492279e-01
 -8.32096636e-02  3.49241719e-02  3.29786278e-02 -2.30267435e-01
  4.28695530e-02 -1.88279077e-01 -1.26993969e-01 -6.13640733e-02
  7.61187971e-02 -4.54428111e-04  2.97921598e-01 -2.75350302e-01
  1.38387773e-02  1.77264273e-01 -1.13952823e-01 -2.93140620e-01
  1.81758478e-01  3.54924351e-02 -2.00587571e-01  1.06810391e-01
 -4.38253395e-02 -2.26804335e-03  2.44872004e-01 -3.01551163e-01
 -1.92054778e-01 -8.62731487e-02 -1.99527189e-01 -1.63892731e-01
  1.33257985e-01 -1.76515244e-03  9.17413309e-02 -3.56211632e-01
 -3.27521414e-01 -1.00153841e-01 -3.81238222e-01  1.63100027e-02
 -1.85901105e-01 -2.80802518e-01  5.09365015e-02  4.00341868e-01
  1.49068311e-01  1.15103155e-01 -7.69534782e-02 -2.73080319e-02
  6.64441958e-02  3.09721362e-02 -9.23274606e-02 -1.89883232e-01
 -5.33280484e-02 -2.77279392e-02 -2.94274092e-01  4.53651231e-03
 -8.55516717e-02  4.52399403e-01  2.37562716e-01  1.20